In [1]:
import tweepy
import pandas as pd
import time
import GetOldTweets3 as got

In [2]:
consumer_key = "l5QjEyheo2lkqWmt6SCAwdrll"
consumer_secret = "SI0ceaSqKi4KJurVjLIExWMEXHEU2TiGzzKEhXBuo6LIwWaFEP"
access_token = "1312660164135194625-BhA3rY2jyPH6Hb2IXBWkQQiuvyc5nz"
access_token_secret = "0pYb0UVtMg5ukg8VQxO0RE6DqnwcDTvnVGZTqiv0pVn1E"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [189]:
import twint
import nest_asyncio
nest_asyncio.apply()

c = twint.Config() # twint config 선언

# Parameter setting
c.Limit = 100
c.Search = '집값'
c.Since = '2019-01-01'
c.Until = '2019-01-10'
c.Output = 'price_sample_tweet2019.json'
c.Popular_tweets = True
c.Store_json = True
c.Hide_output = True


In [190]:
twint.run.Search(c)

In [191]:
import pandas as pd

df = pd.read_json('price_sample_tweet2019.json', lines=True)
dataset=df[["id", "tweet", "retweets_count"]]
dataset=dataset[:100]

In [192]:
dataset.shape

(100, 3)

In [193]:
dataset.columns=['id','document','label']

In [194]:
dataset_test= dataset

In [40]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import gluonnlp as nlp
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import urllib.request

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import random
import time
import datetime

In [9]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
GeForce GTX 1060 6GB


In [10]:
#배치 및 데이터로더 설정

BATCH_SIZE = 1


In [195]:
#테스트 셋에도 똑같이 적용
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
MAX_LEN = 128

sentences = dataset_test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = dataset_test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
################################################################################################

In [12]:
#모델 학습

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: GeForce GTX 1060 6GB


In [15]:
model = torch.load('C:\\Users\\AAA\\Desktop\\dev\\트위터 api\\modeltrained.pt')

model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [16]:
#accuracy와 시간함수

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [196]:
t0 = time.time()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
lab=[]

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    out= logits.flatten().tolist()
    lab.append(out.index(max(out)))
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

print(lab)


Accuracy: 0.33
Test took: 0:00:02
[0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0]


In [197]:
dataset_test['predicted']= lab
dataset_test.head(49)

,id,document,label,predicted
0,1083142372655955969,앑) 그때 집값이 많이 올랐지...,0,0
1,1083139609477804032,코딱지만한 돈으로 쉽게 구해지겠어요? 반지하에서 계단 몇 개 올라가는데 집값은 초고...,0,0
2,1083138836585078784,file:///C:/Users/jmkim/Documents/블로그한글%20조인스/0...,0,1
3,1083124871607967744,집도 구해야 하는데 일도 해야하고 우주도...공부도 등록금도 집값도 여행도 다 자신...,0,0
4,1083081741621911553,"야, 이거 그래도 히스토릭 서비스 반장까지 했는데 집이 뭐 이 모양이냐. 이 동네 ...",0,1
5,1083072760975450113,@alldrinaforth 감사합니다. 다행이 저희 동네는 집은 많아서 좋습니다. ...,0,0
6,1083053813836132352,볼려고 찍어놨던 집들이 하나 둘 집값을 내린다. 크게는 거의 10%까지. 더 기달려...,0,1
7,1083036962557091840,D - 40 드디어 40일 남았어여요 하루하루가 고비지만 나름 잘 참고있어요 40일...,0,1
8,1083029805098926081,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ집단살해 사건 있던 건물로 이사 왔냐...,0,1
9,1083026995640819712,@etwewzz @etwewzz 그 드라마가 그래서 부모들 사회경제적 계층이 애매하...,0,1


In [198]:
pos= 0
neg=0

for item in dataset_test['predicted']:
    if item==1:
        pos+=1
    else:
        neg+=1
        
        
print(pos, neg)

60 40
